<a href="https://colab.research.google.com/github/Xasan6988/chat_bot/blob/main/Chat_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random
import nltk
import json
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

import logging

from telegram import Update, ForceReply
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext


In [2]:
with open('/content/BOT_CONFIG.json', 'r') as f:
  BOT_CONFIG = json.load(f)

# with open('/content/BOT_CONFIG.json', 'w') as f:
#   json.dump(BOT_CONFIG, f, ensure_ascii=false, indent=3)


## День 1


In [ ]:
def clean(text):
  output_text = ''
  for char in text.lower():
    if char in 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя':
      output_text = output_text + char
  return output_text

def get_intent(text): 
  for intent in BOT_CONFIG['intents']:
    for example in BOT_CONFIG['intents'][intent]['examples']:
      text1 = clean(example)
      text2 = clean(text)
      if len(text1) != 0 and len(text2) != 0:
        if nltk.edit_distance(text1, text2) / max(len(text1), len(text2)) < 0.4:
          return intent
  return 'Не удалось определить интент'

In [ ]:
def bot(input_text):
  intent = get_intent(input_text)
  if intent != 'Не удалось определить интент':
    return random.choice(BOT_CONFIG["intents"][intent]['responces'])
  else:
    return 'Извините, я ничего не понял. Я просто глупый бот.'


In [ ]:
input_text = ''
while input_text != 'Завершить работу':
  input_text = input()
  print(bot(input_text))


## День 2


In [ ]:
X = [] 
y = []
for intent in BOT_CONFIG['intents']:
      try:
        if intent != 'other':
          for example in BOT_CONFIG['intents'][intent]['examples']:
            X.append(example)
            y.append(intent)
      except:
        print(BOT_CONFIG['intents'][intent])

In [ ]:
len(X)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

In [6]:
vectorizer = CountVectorizer(ngram_range=(2, 4), analyzer='char')
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)
len(vectorizer.get_feature_names())


6789

In [ ]:
clf = LogisticRegression(random_state=0).fit(X_train_vectorized, y_train)
# clf = RandomForestClassifier(n_estimators=200).fit(X_train_vectorized, y_train)

In [ ]:
clf.score(X_train_vectorized, y_train)
# 0.859704641350211 LogReg_base

In [ ]:
clf.score(X_test_vectorized, y_test)
# 0.1729957805907173 LogReg_base

In [ ]:
clf.predict(vectorizer.transform(['Откуда ты?']))

In [11]:
def get_intent_by_model(text):
  return clf.predict(vectorizer.transform([text]))[0]

## Вызов бота

In [12]:
def bot(input_text):
  intent = get_intent_by_model(input_text)
  return random.choice(BOT_CONFIG["intents"][intent]['responses'])

In [ ]:
input_text = ''
while input_text != 'Завершить работу':
  input_text = input()
  print(bot(input_text))


## День 3. Подключение бота к телеграм

In [ ]:
# !pip install python-telegram-bot --upgrade

In [13]:
# Enable logging
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO
)

logger = logging.getLogger(__name__)


# Define a few command handlers. These usually take the two arguments update and
# context.
def start(update: Update, context: CallbackContext) -> None:
    """Send a message when the command /start is issued."""
    user = update.effective_user
    update.message.reply_markdown_v2(
        fr'Hi {user.mention_markdown_v2()}\!',
        reply_markup=ForceReply(selective=True),
    )


def help_command(update: Update, context: CallbackContext) -> None:
    """Send a message when the command /help is issued."""
    update.message.reply_text('Help!')


def echo(update: Update, context: CallbackContext) -> None:
    """Echo the user message."""
    update.message.reply_text(bot(update.message.text))


def main() -> None:
    """Start the bot."""
    # Create the Updater and pass it your bot's token.
    updater = Updater("")

    # Get the dispatcher to register handlers
    dispatcher = updater.dispatcher

    # on different commands - answer in Telegram
    dispatcher.add_handler(CommandHandler("start", start))
    dispatcher.add_handler(CommandHandler("help", help_command))

    # on non command i.e message - echo the message on Telegram
    dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, echo))

    # Start the Bot
    updater.start_polling()

    # Run the bot until you press Ctrl-C or the process receives SIGINT,
    # SIGTERM or SIGABRT. This should be used most of the time, since
    # start_polling() is non-blocking and will stop the bot gracefully.
    updater.idle()

In [ ]:
main()